In [1]:
import pandas as pd
import numpy as np
import math
from scipy import stats
import os
import sys
from scipy.stats import entropy
from spellchecker import SpellChecker
import matplotlib.pyplot as plt
from bokeh.plotting import figure, output_file, save
from bokeh.models import ColumnDataSource
from bokeh.embed import file_html
from bokeh.resources import CDN
from bokeh.transform import factor_cmap
from bokeh.palettes import Spectral6



In [2]:
df = pd.read_csv('mngr.csv')

In [3]:
'''
file type detectio algo 

input = data path
output = classification [structured, unstructured], extension = [xls, csv, jpeg, etc.]


'''
def check_input(input_path):
    
    #Global variable to store data type and format 
    dict = {}
    
    # Checking if the input path exists
    if not os.path.exists(input_path):
        print(f"Error: {input_path} does not exist")
        return

    # Checking if the input path is a file
    if os.path.isfile(input_path):
        # Check if the input is structured 
        file_extension = os.path.splitext(input_path)[1]
        if file_extension in ['.csv', '.xlsx','xls']:
            dict['classification'] = 'structured'
            dict['extension'] = file_extension
            return dict
            
        #Checking in the input is unstructured    
        elif file_extension in ['.txt']:
            dict['classification'] = 'unstructured'
            dict['extension'] = file_extension
            return dict
            
        #Checking is the input is semi-structured    
        elif file_extension in ['.json']:
            dict['classification'] = 'semi-structured'
            dict['extension'] = file_extension
            return dict
        
        else:
            dict['classification'] = 'unknown'
            dict['extension'] = file_extension
            return dict
            #print(f"{input_path} does not fit any current criteria")
            
    # Check if the input path is a directory
    elif os.path.isdir(input_path):
        # Check if the directory contains images or videos
        contains_images = False
        contains_videos = False
        for filename in os.listdir(input_path):
            if filename.endswith(('.jpg', '.jpeg', '.png', '.gif')):
                contains_images = True
                
            elif filename.endswith(('.mp4', '.avi', '.mov', '.wmv')):
                contains_videos = True
                
        if contains_images and contains_videos:
            dict['classification'] = 'unstructured'
            dict['extension'] = file_extension
            return dict
           
        elif contains_images:
            dict['classification'] = 'unstructured'
            dict['extension'] = file_extension
            return dict
            
        elif contains_videos:
            dict['classification'] = 'unstructured'
            dict['extension'] = file_extension
            return dict
            
        else:
            print(f"{input_path} is an empty directory")
            
    else:
        print(f"{input_path} is not a file or directory")





## Structured Data Metadata Generation Blocks

#### Nulls M1

In [4]:
'''
Input:
A pandas dataframe df

Output:

A new pandas dataframe with the same shape as the input df, where every cell that is a null is populated with 
a 1, and every other cell with a 0.


Function description:
The nulls_value function takes in a pandas dataframe and returns a new dataframe of the same shape 
where each cell containing a null value (as defined in the function) is replaced with a 1, and every other cell is 
replaced with a 0. The function first defines a nested helper function to determine if a given cell value is null. 
It then applies this helper function element-wise to all cells in the input dataframe using the applymap method. 
Finally, the function returns the resulting new dataframe.

Input:
   A     B     C    D
0  1   NaN   3.0   aa
1  4   5.0   6.0  NaN
2  7   8.0   NaN   bb
3  9  10.0  11.0   cc

Output:
   A  B  C  D
0  0  1  0  0
1  0  0  0  1
2  0  0  1  0
3  0  0  0  0

'''
def nulls_value(df):
    def is_null(val):
        if isinstance(val, str) and val.lower() in ['null', 'nan', 'n/a', 'na' , '']:
            return 1
        elif isinstance(val, float) and np.isnan(val):
            return 1
        else:
            return 0
        
    output_df = df.applymap(is_null)
    return output_df

#### Value Length M2

In [5]:
'''
Input:

A pandas dataframe df

Output:

A new pandas dataframe with the same shape as the input df, where each cell is replaced by the length 
of the value populating that cell.

Function description:
The value_length function takes in a pandas dataframe and returns a new dataframe with the same shape where 
each cell is replaced by the length of the value populating that cell. The function first defines a nested 
helper function to get the length of the string representation of a given cell value. It then applies this 
helper function element-wise to all cells in the input dataframe using the applymap method. Finally, the 
function returns the resulting new dataframe.


Input:
    col1  col2  col3
0   123   abc   True
1  3.14  None  None
2   def   NaN   456


output:
    col1  col2  col3
0     3     3     4
1     4     0     0
2     3     3     3

'''


def value_length(df):
    def get_value_length(val):
        return len(str(val))

    output_df = df.applymap(get_value_length)
    return output_df

#### Size M3

In [6]:
'''

Input = dataframe(df) of shape n*m
Output = dataframe of shape n*m

Operations:
Calculates size of each cell in bytes. 

'''

def cell_size(df):
    
    # Blank dictionary to story cell size info
    cell_size_info = {}
    
    # Iterating through each column and row to get the cell size information
    for col in df.columns:
        cell_size = []
        for val in df[col]:
            cell_size.append(sys.getsizeof(val))
        cell_size_info[col] = cell_size
    
    # Create a DataFrame from the cell size information
    df_cell_sizes = pd.DataFrame(cell_size_info)

    return df_cell_sizes

#### Data Type M4

In [7]:
def determine_data_type(df):
    '''
    This function takes in a pandas dataframe and returns a new dataframe with the same shape
    where each cell is replaced by the data type of the value populating that cell.
    
    Input:
        - df: pandas dataframe
        
    Output:
        - A new pandas dataframe with the same shape as the input dataframe, where each cell 
          is replaced by the data type of the value populating that cell.
    
    
    
    Input:
         col1  col2  col3
    0   123   abc   True
    1  3.14  None  None
    2   def   NaN   456
    
    
    
    Output:
        col1     col2       col3
    0    int      str       bool
    1  float  unknown    unknown
    2    str  unknown      int64

    '''
    output_df = pd.DataFrame()

    for col in df.columns:
        output_df[col] = df[col].apply(lambda x: 'unknown' if pd.isna(x) else type(x).__name__)
    
    return output_df

#### Value Count M5

In [8]:
def value_count(df):
    '''
    This function takes in a pandas dataframe and returns a dictionary for each column,
    where the keys of the dictionary are unique values in that column, and the values 
    are the number of occurrences of each unique value.
    
    Input:
        - df: pandas dataframe
        
    Output:
        - A dictionary of value counts for each column in the input dataframe.
        
        
    input:
        col1 col2 col3
    0     1    A    X
    1     2    B    Y
    2     2    A    X
    3     1    B    Z
    4     3    C    Y
    5     1    C    Z
    6     2    A    Z
    7     2    B    X


    output:
    {
        'col1': {1: 3, 2: 4, 3: 1},
        'col2': {'A': 3, 'B': 2, 'C': 2},
        'col3': {'X': 3, 'Y': 2, 'Z': 3}
    }
    '''
    output_dict = {}

    for col in df.columns:
        value_counts = df[col].value_counts().to_dict()
        output_dict[col] = value_counts
    
    return output_dict

#### Z Score (Numeric) M6

In [9]:
def z_score(df):
    '''
    This function takes in a pandas dataframe and calculates the z-scores of each
    value in each numeric column, returning a new dataframe with the same shape as
    the input dataframe, but with the values of each numeric column replaced by
    their z-scores within the column.

    Input:
        - df: pandas dataframe

    Output:
        - A new pandas dataframe with the same shape as the input dataframe, but with
          the values of each numeric column replaced by their z-scores within the column.

    Input:
      col1  col2  col3
    0     1   100     5
    1     2   200    10
    2     3   300    15
    3     4   400    20
    4     5   500    25

    Output:
          col1      col2
    0 -1.414214 -1.414214
    1 -0.707107 -0.707107
    2  0.000000  0.000000
    3  0.707107  0.707107
    4  1.414214  1.414214
    '''
    df_copy = df.copy()

    # Convert 'object' datatype columns containing numeric data to the appropriate numeric datatype
    for col in df_copy.columns:
        if df_copy[col].dtype == 'object':
            df_copy[col] = pd.to_numeric(df_copy[col], errors='ignore')

    numeric_cols = df_copy.select_dtypes(include=['float', 'int', 'float64', 'float32', 'float16']).columns
    zscore_df = df_copy[numeric_cols].copy()
    
    for col in numeric_cols:
        #if df_copy[col].isnull().all():
        #    print('in')
        #    continue  # skip the column if it has all nulls
        zscores = stats.zscore(df_copy[col], nan_policy='omit')
        zscore_df[col] = zscores
    
    
    zscore_df = zscore_df.dropna(how='all', axis=1)
    return zscore_df


#### Z Score (Value Length) M7

In [10]:
def z_score_value_length(df):
    
    '''
    Value Lenght Z Score for Non Numeric Data
    
    This function takes in a pandas dataframe and calculates the z-scores of each
    cell value length in each non-numeric column, returning a new dataframe with the
    same shape as the input dataframe, but with the cell values of each non-numeric column
    replaced by their z-scores based on the length of the cell value.
    
    Input:
        - df: pandas dataframe
        
    Output:
        - A new pandas dataframe with only the non-numeric columns and with the same shape
          as the input dataframe, but with the cell values of each non-numeric column
          replaced by their z-scores based on the length of the cell value.
          
          
    Input:
        col1  col2    col3  col4
    0  abcd   1.2   apple     5
    1    ab   2.4  banana     6
    2  abcde   3.6   peach     7
    3   abc   4.8  orange     8


    Output:

            col1      col3
    0  -0.205583 -0.872872
    1  -1.468296  0.218218
    2   1.555266  1.309307
    3  -0.881387 -0.654653
    '''
    non_numeric_cols = df.select_dtypes(exclude=['float', 'int']).columns
    zscore_df = pd.DataFrame(index=df.index)

    for col in non_numeric_cols:
        if not df[col].isnull().all():
            value_lengths = df[col].astype(str).apply(len)
            zscores = stats.zscore(value_lengths)
            zscore_df[col] = zscores
    
    return zscore_df


#### Decimals M8

In [11]:
def calculate_decimal_values(df):
    '''
    This function takes in a pandas dataframe and returns a new dataframe with only the numeric columns,
    where the cell values are replaced with the number of decimal places of the corresponding values.
    
    Input:
        - df: pandas dataframe
        
    Output:
        - A new pandas dataframe with only the numeric columns of the input dataframe,
          where the cell values are replaced with the number of decimal places of the corresponding values.
          
    Input:          
        col1    col2     col3   col4
    0     1  1.2000  1.00000      A
    1     2  2.3400  2.00000     BB
    2     3  3.4560  3.00000    CCC
    3     4  4.5678  4.00000   DDDD
    4     5  5.6789  5.00000  EEEEE


    Output:
        col2  col3
    0     1     0
    1     2     0
    2     3     4
    3     4     5
    4     5     5
    '''
    # Filter the numeric columns in the input dataframe
    num_cols = df.select_dtypes(include=['float', 'int']).columns
    
    # Create a new dataframe to store the decimal place counts for each cell value
    decimal_count_df = pd.DataFrame(index=df.index, columns=num_cols)
    # Check if there are any numeric columns in the input dataframe
    #print(len(decimal_count_df))
    if len(num_cols) == 0:
        # If there are no numeric columns, return an empty dataframe with the same index as the input dataframe
        return pd.DataFrame(index=df.index)
    # Loop through each numeric column in the input dataframe
    # Loop through each numeric column in the input dataframe
    for col in num_cols:
        try:
            # Convert the column to a string data type and split each cell value at the decimal point
            decimal_parts = df[col].astype(str).str.split('.', expand=True)
            # Calculate the number of decimal places for each cell value and assign it to the corresponding cell 
            # in the output dataframe
            decimal_count_df[col] = decimal_parts[1].str.len().fillna(0).astype(int)
        except:
            # If an exception is raised during the loop, set the corresponding column in the output dataframe to null
            decimal_count_df[col] = np.nan
    
    
    return decimal_count_df


### Column Metadata

#### Inconsistency M9

In [12]:
def count_elements(lst):
    counts = {}
    for elem in lst:
        if elem in counts:
            counts[elem] += 1
        else:
            counts[elem] = 1
    return [counts[elem] for elem in lst]


def column_inconsistency(col):
    # check if column is numeric
    if not pd.api.types.is_numeric_dtype(col):
        col = count_elements(col)

    # calculate the standard deviation of the column
    std_dev = np.std(col)

    # calculate the inconsistency
    inconsistency = std_dev / np.mean(col)

    return inconsistency


def get_column_inconsistency(df):
    """
    This function takes a pandas dataframe as input and returns a dataframe with two columns - column name and inconsistency.
    Higher inconsistency values indicate higher inconsistencies in the column data.

    Inputs:
    A pandas dataframe with any number of columns and rows.

    Outputs:
    A pandas dataframe with two columns - column name and inconsistency.
    
    
    Input:
        name  age gender
    0  Alice   25      F
    1    Bob   30      M
    2   Charlie  35      M
    
          Column  inconsistency
    0           name       0.000000
    1            age       0.057982
    2         gender       0.408248
    
    """
    inconsistency_dict = {}
    for col in df.columns:
        inconsistency_dict[col] = column_inconsistency(df[col])

    return pd.DataFrame({'Column': list(inconsistency_dict.keys()), 'inconsistency': list(inconsistency_dict.values())})


#### Entropy M10

In [13]:
def entropy_of_df(df):
    """
    This function takes in a pandas dataframe and returns a dictionary with column names as keys and entropy as values.

    Inputs:
    A pandas dataframe with any number of columns and rows.

    Outputs:
    A dictionary with column names as keys and entropy as values.
    
    Important notes:
    In the context of this function, a high entropy value for a column means that the values in that column are 
    diverse and spread out, with no single value being dominant or occurring significantly more frequently than 
    others. This can be an indication that the column contains important information and could potentially be a 
    good predictor in a machine learning model. On the other hand, a low entropy value for a column means that the 
    values in that column are more similar and possibly redundant, which may not provide much information gain in a 
    machine learning model.
    
    """
    entropies = []
    for col in df.columns:
        if pd.api.types.is_numeric_dtype(df[col]) and not df[col].isnull().all():
            # if column is numeric and not all NaN, use the probability density function to calculate the entropy
            pdf, _ = np.histogram(df[col].dropna(), density=True)
            entropies.append({'Column': col, 'Entropy': entropy(pdf, base=2)})
        else:
            # if column is not numeric or contains only NaN values, use value_counts to calculate the entropy
            counts = df[col].value_counts(normalize=True)
            entropies.append({'Column': col, 'Entropy': entropy(counts, base=2)})
    return pd.DataFrame(entropies)


#### Constancy M11

In [14]:
def get_top_five_frequent_values(df):
    """
    This function takes a pandas dataframe as input and returns a dictionary with each column's name as a key and the 5 most frequent values of that column as values. 

    Inputs:
    A pandas dataframe with columns of any data type.

    Outputs:
    A dictionary with column names as keys and a list of 5 most frequent values as values.
    
    
    
    Input:
        col1  col2  col3
    0     a     1     x
    1     a     1     y
    2     b     2     y
    3     c     2     y
    4     c     3     z
    5     c     3     z


    Output
    {
        'col1': ['c', 'a', 'b'],
        'col2': [3, 2, 1],
        'col3': ['y', 'z', 'x']
    }

    """
    top_five_dict = {}
    for col in df.columns:
        top_five_dict[col] = list(df[col].value_counts().head(5).index)
    return top_five_dict

#### Numer of Rows M12

In [15]:
def get_column_row_counts(df):
    """
    This function takes a pandas dataframe as input and returns a dictionary with each column's name as a key and its row count as the value.

    Inputs:
    A pandas dataframe with columns of any data type.

    Outputs:
    A dictionary with column names as keys and number of rows as values.
    
    
    Input
       col1  col2  col3
    0     a     1     x
    1     a     NaN   y
    2     b     2     y
    3     NaN   2     y
    4     c     3     z
    5     c     3     NaN


    Output
    {
        'col1': 4,
        'col2': 5,
        'col3': 4
    }
    
    """
    column_row_counts = {}
    for col in df.columns:
        column_row_counts[col] = df[col].count()
    return column_row_counts

#### Numer of Unique Values in Column M13

In [16]:
def get_column_unique_value_counts(df):
    """
    This function takes a pandas dataframe as input and returns a dictionary with each column's name as a key and its unique value count as the value.

    Inputs:
    A pandas dataframe with columns of any data type.

    Outputs:
    A dictionary with column names as keys and number of unique values as values.
    
    
    Input
        col1  col2  col3
    0     a     1     x
    1     a     1     y
    2     b     2     y
    3     c     2     y
    4     c     3     z
    5     c     3     z


    Output
    {
        'col1': 3,
        'col2': 3,
        'col3': 3
    }
    
    """
    column_unique_counts = {}
    for col in df.columns:
        column_unique_counts[col] = df[col].nunique()
    return column_unique_counts

#### Numer of Duplicated Values in Column M14

In [17]:
def get_column_duplicate_counts(df):
    """
    This function takes a pandas dataframe as input and returns a dictionary with each column's name as a key and its duplicate value count as the value.

    Inputs:
    A pandas dataframe with columns of any data type.

    Outputs:
    A dictionary with column names as keys and number of duplicates as values.
    
    
    Input
        col1  col2  col3
    0     a     1     x
    1     a     1     y
    2     b     2     y
    3     c     2     y
    4     c     3     z
    5     c     3     z

    Output
    {
        'col1': 2,
        'col2': 3,
        'col3': 3
    }
    
    """
    
    column_duplicate_counts = {}
    for col in df.columns:
        column_duplicate_counts[col] = sum(df[col].duplicated())
    return column_duplicate_counts

#### Datatype Distribution M15

In [18]:
def get_column_datatype_percentage(df):
    """
    This function takes a pandas dataframe as input and returns a dictionary with each column's name as a key and a nested dictionary as the value. In the nested dictionary, the keys are the data types present in that column and the values are the percentage of occurrences of each data type, including those with 0% occurrence.

    Inputs:
    A pandas dataframe with columns of any data type.

    Outputs:
    A dictionary with column names as keys and a nested dictionary as the value.
    
    
    Input:
         col1  col2  col3
    0     a     1     True
    1     b     2     False
    2     c     NaN   True
    3     d     4     True
    4     e     NaN   False

    Output:
    {
        'col1': {'str': 100.0, 'bool': 0.0, 'float': 0.0, 'unknown': 0.0},
        'col2': {'int': 50.0, 'float': 50.0, 'bool': 0.0, 'unknown': 0.0},
        'col3': {'bool': 60.0, 'unknown': 40.0, 'int': 0.0, 'float': 0.0}
    }

    """
    
    column_datatype_percentage = {}
    for col in df.columns:
        datatype_counts = determine_data_type(df[[col]]).iloc[:, 0].value_counts()
        datatype_percentage = datatype_counts / datatype_counts.sum() * 100
        nested_dict = {k: v for k, v in zip(datatype_percentage.index, datatype_percentage)}
        for datatype in set(determine_data_type(df[[col]]).iloc[:, 0].unique()) - set(datatype_percentage.index):
            nested_dict[datatype] = 0
        nested_dict_sorted = dict(sorted(nested_dict.items(), key=lambda item: item[1], reverse=True))
        column_datatype_percentage[col] = nested_dict_sorted
    return column_datatype_percentage

### Dataset Level Metadata

#### Numer of Rows in Dataset M16

In [19]:
def get_row_count(df):
    """
    This function takes a pandas dataframe as input and returns the number of rows in that dataframe.

    Inputs:
    A pandas dataframe with any number of rows.

    Outputs:
    An integer representing the number of rows in the input dataframe.
    
    
    Input:
        col1  col2  col3
    0     1     a     True
    1     2     b     False
    2     3     c     True
    3     4     d     False
    4     5     e     True

    Output:
    5

    """
    return df.shape[0]

#### Numer of Unique Rows in Dataset M17

In [20]:
def get_unique_row_count(df):
    """
    This function takes a pandas dataframe as input and returns the number of unique rows in that dataframe.

    Inputs:
    A pandas dataframe with any number of rows.

    Outputs:
    An integer representing the number of unique rows in the input dataframe.
    
    input:
        col1  col2  col3
    0     1     a     True
    1     2     b     False
    2     3     c     True
    3     4     d     False
    4     5     e     True

    output:
    5

    """
    return df.drop_duplicates().shape[0]

#### Numer of Duplicates in Dataset M18

In [21]:
def get_duplicate_row_count(df):
    """
    This function takes a pandas dataframe as input and returns the number of duplicate rows in that dataframe.

    Inputs:
    A pandas dataframe with any number of rows.

    Outputs:
    An integer representing the number of duplicate rows in the input dataframe.
    
    Input:
        col1  col2  col3
    0     1     a     True
    1     2     b     False
    2     3     c     True
    3     4     d     False
    4     5     e     True
    5     3     c     True
    6     6     f     False

    Output:
    1

    """
    return df.duplicated().sum()

### Conditions 

#### Missing 

In [22]:
import pandas as pd

def missing_c1(df):
    """
    This function takes a pandas dataframe as input and returns a dataframe with two columns:
    col1 = name of column in input df
    col2 = percentage of missing values in that column

    Inputs:
    A pandas dataframe with any number of rows and columns.

    Outputs:
    A pandas dataframe containing the percentage of missing values in each column, sorted by percentage of missing values in descending order.
    """
    # Compute the percentage of missing values in each column
    missing_percents = df.isnull().mean() * 100

    # Create a dataframe with the column names and their corresponding percentage of missing values
    missing_df = pd.DataFrame({'Column Name': missing_percents.index, 'Missing %': missing_percents.values})

    # Sort the dataframe by the percentage of missing values in descending order
    missing_df = missing_df.sort_values(by='Missing %', ascending=False)

    return missing_df


In [23]:
import pandas as pd

def col_duplicate_c2(df):
    """
    This function takes a pandas dataframe as input and returns two outputs.
    Output1 is a pandas dataframe with two columns representing the column names in the original dataframe and 
    the percentage of duplicates in that column, sorted in descending order.
    Output2 is an integer representing the number of duplicate rows in the whole dataset.

    Inputs:
    A pandas dataframe with any number of rows and columns.

    Outputs:
    Output1 - A pandas dataframe with two columns and a single row for each column in the original dataframe that 
    has duplicate values. Column 1 represents the column names in the original dataframe. Column 2 represents 
    the percentage of duplicates in that column, sorted in descending order.
    Output2 - An integer representing the number of duplicate rows in the whole dataset.
    """

    # Calculate the percentage of duplicates for each column in the dataframe
    duplicate_percentage = []
    for col in df.columns:
        num_duplicates = df.duplicated(subset=[col]).sum()
        col_percentage = (num_duplicates / df.shape[0]) * 100
        if col_percentage > 0:
            duplicate_percentage.append([col, col_percentage])
    if len(duplicate_percentage) > 0:
        # Create a pandas dataframe with the column names and their respective duplicate percentages
        df_duplicate_percentage = pd.DataFrame(duplicate_percentage, columns=['Column Name', 'Duplicate Percentage'])
        # Sort the dataframe in descending order based on the Duplicate Percentage column
        df_duplicate_percentage = df_duplicate_percentage.sort_values('Duplicate Percentage', ascending=False)
    else:
        df_duplicate_percentage = pd.DataFrame(columns=['Column Name', 'Duplicate Percentage'])
    # Get the number of rows in the whole dataset that are duplicates
    num_rows_duplicates = df[df.duplicated()].shape[0]
    # Return the pandas dataframe with the duplicate percentages and the number of rows in the whole dataset that are duplicates
    return df_duplicate_percentage, num_rows_duplicates


In [24]:
from spellchecker import SpellChecker
import pandas as pd

def misspelled(df):
    """
    This function takes a pandas dataframe as input and returns a tuple containing a list of column names with misspelled
    words and a count of total misspelling occurrences.

    Inputs:
    A pandas dataframe with any number of rows and columns.

    Outputs:
    A tuple containing a list of strings representing the names of columns with misspelled words and an integer
    representing the count of total misspelling occurrences.
    """

    # Initialize a SpellChecker instance with the default language
    spell = SpellChecker()

    # Initialize an empty list to store column names with misspelled words
    misspelled_cols = []

    # Initialize a count of total misspellings
    total_misspellings = 0

    # Loop through each column in the input dataframe
    for col in df.columns:
        # Initialize a count of misspellings in the current column
        col_misspellings = 0

        # Loop through each cell in the current column
        for val in df[col]:
            # Check if the cell value is a string of length >= 2 and contains any misspelled words
            if isinstance(val, str) and len(val) >= 2:
                # Check if the cell value is not a number, time, or date
                try:
                    float_val = float(val)
                    if float_val == int(float_val):
                        val = int(float_val)
                    else:
                        continue
                except ValueError:
                    pass
                if ":" in val or "-" in val or "/" in val:
                    continue
                # Check if the cell value contains multiple words
                if " " in val:
                    # Split the cell value into words and check each word for misspellings
                    words = val.split()
                    for word in words:
                        if not spell.correction(word) == word:
                            col_misspellings += 1
                            total_misspellings += 1
                # Check if the cell value is a single word and is misspelled
                elif not spell.correction(val) == val:
                    col_misspellings += 1
                    total_misspellings += 1

        # If the current column has at least one misspelled word, add its name to the list of misspelled columns
        if col_misspellings > 0:
            misspelled_cols.append(col)

    # Return a tuple containing the list of misspelled columns and the total count of misspellings
    return misspelled_cols, total_misspellings


In [25]:
def free_form_text(df):
    # Determine data types of each cell in the input dataframe
    data_types_df = determine_data_type(df)

    # Get the length of each cell value in the input dataframe
    value_lengths_df = value_length(df)

    # Find cells with free-form text
    free_form_mask = (data_types_df == 'str') & (value_lengths_df >= 50)

    # Calculate percentage of rows with free-form text for each column
    percentage_df = free_form_mask.apply(lambda x: round(sum(x) / len(x) * 100, 2))

    # Combine column names and corresponding percentages in a new dataframe
    output_df = pd.DataFrame({'Column Name': percentage_df.index, 'Percentage of Rows with Free-form Text': percentage_df.values})

    # Sort the rows of the output dataframe in descending order
    output_df = output_df.sort_values(by='Percentage of Rows with Free-form Text', ascending=False)

    return output_df


In [26]:
def datatype_mismatch(df):
    # Determine data types of each cell in the input dataframe
    data_types_df = determine_data_type(df)

    # Determine the recommended datatype for each column
    recommendation_df = pd.DataFrame(columns=['Column Name', 'Column Datatype', 'Recommended Datatype', 'Percentage of Recommended Cells'])

    for col in data_types_df.columns:
        # Determine the column datatype
        col_dtype = df[col].dtype

        # Calculate the percentage of cells with each datatype in the column
        datatype_percentages = data_types_df[col].value_counts(normalize=True, dropna=False)

        # Determine the most common datatype in the column
        if (datatype_percentages.index[0] == 'unknown') and (len(datatype_percentages) < 2):
            continue
        elif (datatype_percentages.index[0] == 'unknown') and (len(datatype_percentages) > 1):
            majority_datatype = datatype_percentages.index[1]
        else:
            majority_datatype = datatype_percentages.index[0]

        # Determine the percentage of cells with the most common datatype
        if majority_datatype not in str(col_dtype):
            percentage = round(datatype_percentages[majority_datatype] * 100, 2)
        else:
            continue

        # Append the column name, column datatype, recommended datatype, and percentage to the recommendation dataframe
        recommendation_df = pd.concat([recommendation_df, pd.DataFrame({'Column Name': [col], 'Column Datatype': [col_dtype],
                                                                         'Recommended Datatype': [majority_datatype],
                                                                         'Percentage of Recommended Cells': [percentage]})],
                                      ignore_index=True)
    
    
    recommendation_df['Column Name'] = recommendation_df['Column Name'].astype(str)
    
    # Sort the dataframe by 'Percentage of Recommended Cells' in descending order
    recommendation_df = recommendation_df.sort_values(by=['Percentage of Recommended Cells'], ascending=False)

    
    return recommendation_df


In [56]:
def column_outliers(df):
    """
    This function takes a pandas dataframe as input and returns a dataframe with column names and
    the percentage of outliers in each column. If the column is numeric, z-score based outliers
    are used. If the column is non-numeric, z-score based outliers based on cell value length are used.

    Inputs:
    A pandas dataframe with any number of columns and rows.

    Outputs:
    A dataframe with two columns: the first column contains column names, and the second column contains
    the percentage of outliers in each column.
    """

    # Define the helper functions
    def column_outliers_numeric(num_z):
        zscores = num_z
        outliers = np.abs(zscores) > 3
        return np.mean(outliers)

    def column_outliers_nonnumeric(val_z):
        zscores = val_z
        outliers = np.abs(zscores) > 3
        return np.mean(outliers)

    # Determine column outliers for each column
    outliers_dict = {}
    num_z = z_score(df)
    val_z = z_score_value_length(df)
    for col in df.columns:
        if pd.api.types.is_numeric_dtype(df[col]):
            if (col in num_z.columns):
                outliers_dict[col] = column_outliers_numeric(num_z[col])
        else:
            if (col in val_z.columns):
                outliers_dict[col] = column_outliers_nonnumeric(val_z[col])

    # Convert dictionary to dataframe
    outliers_df = pd.DataFrame(list(outliers_dict.items()), columns=['Column Name', 'Percentage of Outliers'])
    
    # Sort the dataframe by 'Percentage of Recommended Cells' in descending order
    outliers_df = outliers_df.sort_values(by=['Percentage of Outliers'], ascending=False)


    return outliers_df


In [28]:
def misfielded(df):
    def column_outliers_nonnumeric(val_z, df):
        zscores = val_z
        s_sorted = zscores.sort_values(ascending=False)
        # get the fifth biggest value
        p = s_sorted.iloc[6]
        #percentile threshold 
        #p = zscores.quantile(0.999)
        outliers = np.abs(zscores) > p
        return df.where(outliers, other=np.nan) # return actual outlier values from the original DataFrame

    val_z = z_score_value_length(df)
    outliers_list = [column_outliers_nonnumeric(val_z[col], df)[col] for col in val_z.columns] # create list of outlier values for each column
    outliers_df = pd.concat(outliers_list, axis=1) # concatenate outlier values into a single DataFrame
    outliers_df.columns = val_z.columns # set column names to match those in val_z
    outliers_df = outliers_df.dropna(how = 'all', axis=1)
    return outliers_df


In [29]:
def misfielded(df):
    def column_outliers_nonnumeric(val_z, df):
        zscores = val_z
        s_sorted = zscores.sort_values(ascending=False)
        # get the fifth biggest value
        p = s_sorted.iloc[4]
        #percentile threshold 
        #p = zscores.quantile(0.999)
        outliers = np.abs(zscores) > p
        return df.where(outliers, other=np.nan) # return actual outlier values from the original DataFrame

    val_z = z_score_value_length(df)
    outliers_list = [column_outliers_nonnumeric(val_z[col], df)[col] for col in val_z.columns] # create list of outlier values for each column
    outliers_df = pd.concat(outliers_list, axis=1) # concatenate outlier values into a single DataFrame
    outliers_df.columns = val_z.columns # set column names to match those in val_z
    outliers_df = outliers_df.dropna(how = 'all', axis=1)


    input_df = pd.DataFrame(outliers_df)

    def get_compact_dataframe(df):
        compact_df = pd.DataFrame(columns=df.columns)

        for col in df.columns:
            tmp = df.copy()
            non_null_values = (tmp[col].dropna().head(4)).tolist()  # Get up to first 5 non-null values
            compact_df[col] = pd.Series(non_null_values)

        return compact_df

    result_df = get_compact_dataframe(input_df.copy())
    return (result_df)



In [30]:
def incorrect(df):
    
    #t_inc = get_column_inconsistency(df)
    #t_ent = entropy_of_df(df)
    tmp = pd.merge(get_column_inconsistency(df), entropy_of_df(df), on='Column')
    
    i95 = tmp['inconsistency'].quantile(0.95)
    i05 = tmp['inconsistency'].quantile(0.05)

    e95 = tmp['Entropy'].quantile(0.95)
    e05 = tmp['Entropy'].quantile(0.05)


    print('\n \n Flag')
    return (tmp[(tmp['inconsistency']>i95) | (tmp['inconsistency'] < i05) | (tmp['Entropy']>e95) | (tmp['Entropy']<e05)])
    

### Reporting

In [31]:
def missing_r1(missing_df):
    # Truncate long column names
    missing_df['Column Name'] = missing_df['Column Name'].apply(lambda x: x[:65] + '...' if len(x) > 65 else x)
    source = ColumnDataSource(missing_df[:12])
    p = figure(y_range=source.data['Column Name'], x_axis_label='% Missing', y_axis_label='Column Name', title='Percentage of Missing Values by Column', width=800, height=600)
    p.hbar(y='Column Name', right='Missing %', height=0.5, source=source)

    # Save the plot to an HTML file
    output_file('plot.html')
    #save(p)

    # Convert the dataframe to an HTML table
    table_html = missing_df.to_html()

    # Combine the plot and table HTML into a single HTML document
    plot_html = file_html(p, CDN, "plot")
    
    html = f'<div style="display:flex; flex-direction:column; justify-content:center; align-items:center;"><h2 style="text-align:center;">Percentage of Missing Values</h2><div style="text-align:center;">{plot_html}</div><br><div style="text-align:center;">{table_html}</div></div>'

    
    return (html)




In [32]:
def duplicate_r2(df_duplicate_percentage, num_rows_duplicates  ):
    # Create a heading for the HTML element
    heading_html = '<h2 style="text-align:center;">Duplicates</h2>'

    # Create a text for the HTML element
    text_html = f'<p style="text-align:center;">Provided dataset has {num_rows_duplicates} duplicate rows.</p>'

    # Create a horizontal bar chart showing up to 12 columns with the highest percentage of duplicates
    df_duplicate_percentage['Column Name'] = df_duplicate_percentage['Column Name'].apply(lambda x: x[:65] + '...' if len(x) > 65 else x)
    source = ColumnDataSource(df_duplicate_percentage[:12])
    p = figure(y_range=source.data['Column Name'], x_axis_label='% Duplicates', y_axis_label='Column Name', title='Percentage of Duplicates by Column', width=800, height=600)
    p.hbar(y='Column Name', right='Duplicate Percentage', height=0.5, source=source)
    plot_html = file_html(p, CDN, "plot")

    # Convert the output dataframe from the col_duplicate_c2 function to HTML table
    table_html = df_duplicate_percentage.to_html()

    # Combine the heading, text, horizontal bar chart, and output dataframe into a single HTML document
    html = f'<div style="display:flex; flex-direction:column; justify-content:center; align-items:center;">{heading_html}{text_html}<div style="text-align:center;">{plot_html}</div><br><div style="text-align:center;">{table_html}</div></div>'

    # Return the HTML element
    return html

In [33]:
def free_form_text_html(output_df):
    """
    This function takes the output dataframe of the free_form_text function as input and returns an HTML element
    with a heading "Free Form Text in Dataset", a horizontal bar chart showing up to 12 columns with the highest
    percentage of rows with free-form text, and a table displaying all columns with the corresponding percentages
    of rows with free-form text.

    Inputs:
    The output dataframe of the free_form_text function.

    Outputs:
    An HTML element with a heading "Free Form Text in Dataset", a horizontal bar chart showing up to 12 columns with the
    highest percentage of rows with free-form text, and a table displaying all columns with the corresponding percentages
    of rows with free-form text.
    """

    # Truncate column names longer than 65 characters
    output_df['Column Name'] = output_df['Column Name'].apply(lambda x: x[:65] + '...' if len(x) > 65 else x)

    # Create a heading for the HTML element
    heading_html = '<h2 style="text-align:center;">Free Form Text in Dataset</h2>'

    # Create a horizontal bar chart showing up to 12 columns with the highest percentage of rows with free-form text
    source = ColumnDataSource(output_df[:12])
    p = figure(y_range=source.data['Column Name'], x_axis_label='% Rows with Free-form Text', y_axis_label='Column Name', title='Percentage of Rows with Free-form Text by Column', width=800, height=600)
    p.hbar(y='Column Name', right='Percentage of Rows with Free-form Text', height=0.5, source=source)
    plot_html = file_html(p, CDN, "plot")

    # Convert the output dataframe to HTML table
    table_html = output_df.to_html()

    # Combine the heading, horizontal bar chart, and output dataframe into a single HTML document
    html = f'<div style="display:flex; flex-direction:column; justify-content:center; align-items:center;">{heading_html}<div style="text-align:center;">{plot_html}</div><br><div style="text-align:center;">{table_html}</div></div>'

    # Return the HTML element
    return html


In [34]:
def datatype_mismatch_html(output_df):
    """
    This function takes the output dataframe of the datatype_mismatch function as input and returns an HTML element
    with a heading "Datatype Mismatch in Dataset", a horizontal bar chart showing up to 12 columns with the highest
    percentage of rows with datatype mismatch, and a table displaying all columns with the corresponding percentages
    of rows with free-form text.

    Inputs:
    The output dataframe of the datatype mismatch function.

    Outputs:
    An HTML element with a heading "Free Form Text in Dataset", a horizontal bar chart showing up to 12 columns with the
    highest percentage of rows with free-form text, and a table displaying all columns with the corresponding percentages
    of rows with free-form text.
    """
    
    # Truncate column names longer than 65 characters
    output_df['Column Name'] = output_df['Column Name'].apply(lambda x: x[:65] + '...' if len(x) > 65 else x)
    
    tmp = output_df.copy()
    
    output_df = output_df[['Column Name','Percentage of Recommended Cells']]

    # Create a heading for the HTML element
    heading_html = '<h2 style="text-align:center;">Datatype Mismatch in Dataset</h2>'

    # Create a horizontal bar chart showing up to 12 columns with the highest percentage of rows with free-form text
    sub = output_df[['Column Name', 'Percentage of Recommended Cells']]
    source = ColumnDataSource(output_df[:12])
    p = figure(y_range=source.data['Column Name'], x_axis_label='% Rows with Datatype Mismatch', y_axis_label='Column Name', title='Percentage of Rows with Datatype Mismatch by Column', width=800, height=600)
    p.hbar(y='Column Name', right='Percentage of Recommended Cells', height=0.5, source=source)
    plot_html = file_html(p, CDN, "plot")

    # Convert the output dataframe to HTML table
    #table_html = tmp.to_html(index=False)
    table_html = tmp.to_html()

    # Combine the heading, horizontal bar chart, and output dataframe into a single HTML document
    html = f'<div style="display:flex; flex-direction:column; justify-content:center; align-items:center;">{heading_html}<div style="text-align:center;">{plot_html}</div><br><div style="text-align:center;">{table_html}</div></div>'

    # Return the HTML element
    return html


In [35]:
def column_outliers_html(output_df):
    """
    This function takes the output dataframe of the datatype_mismatch function as input and returns an HTML element
    with a heading "Datatype Mismatch in Dataset", a horizontal bar chart showing up to 12 columns with the highest
    percentage of rows with datatype mismatch, and a table displaying all columns with the corresponding percentages
    of rows with free-form text.

    Inputs:
    The output dataframe of the datatype mismatch function.

    Outputs:
    An HTML element with a heading "Free Form Text in Dataset", a horizontal bar chart showing up to 12 columns with the
    highest percentage of rows with free-form text, and a table displaying all columns with the corresponding percentages
    of rows with free-form text.
    """
    
    # Truncate column names longer than 65 characters
    output_df['Column Name'] = output_df['Column Name'].apply(lambda x: x[:65] + '...' if len(x) > 65 else x)

    # Create a heading for the HTML element
    heading_html = '<h2 style="text-align:center;">Percentage of Rows With Outliers</h2>'

    # Create a horizontal bar chart showing up to 12 columns with the highest percentage of rows with free-form text
    source = ColumnDataSource(output_df[:12])
    p = figure(y_range=source.data['Column Name'], x_axis_label='% Percentage of Outliers', y_axis_label='Column Name', title='Percentage of Rows with Outliers', width=800, height=600)
    p.hbar(y='Column Name', right='Percentage of Outliers', height=0.5, source=source)
    plot_html = file_html(p, CDN, "plot")




    # Convert the output dataframe to HTML table
    table_html = output_df.to_html()

    # Combine the heading, horizontal bar chart, and output dataframe into a single HTML document
    html = f'<div style="display:flex; flex-direction:column; justify-content:center; align-items:center;">{heading_html}<div style="text-align:center;">{plot_html}</div><br><div style="text-align:center;">{table_html}</div></div>'

    # Return the HTML element
    return html


In [62]:
def misfielded_html(tmp):
    # Create a heading for the HTML element
    heading_html = '<h2 style="text-align:center;">Potentially Misfielded Values</h2>'
    
    n_cols = tmp.shape[1]

    html_result = ""  # Create an empty string variable
    
    message = "There are {n_cols} columns with potentially misfielded values:\n"


    for col in tmp:
        col_html = pd.DataFrame(tmp[col]).to_html()  # Get HTML for the current column
        html_result += col_html + "\n\n"  # Append the HTML to the existing result with some space between them


    # Concatenate the heading, message, and the HTML results
    final_html = heading_html + message.format(n_cols=n_cols) + html_result
    
    # Wrap the final HTML with a <div> element and set text-align to center
    final_html = f'<div style="text-align:center;">{final_html}</div>'
    
    return final_html


In [106]:
import pandas as pd

def structured_block(path, extension):
    # Check if file extension is csv
    if extension == '.csv':
        df = pd.read_csv(path)
        
    # Check if file extension is xls or xlsx
    elif extension in ['.xls', '.xlsx']:
        df = pd.read_excel(path)
        
    # Handle unrecognized file extension
    else:
        print(f"Error: Unrecognized structured file extension: {extension}")
        df = None
        return df
    
    # Generate HTML for missing values
    missing_df = missing_c1(df)
    html1 = missing_r1(missing_df)
    
    # Generate HTML for duplicates
    dup_df , dup_row_cnt = col_duplicate_c2(df)
    html2 = duplicate_r2(dup_df , dup_row_cnt)
    
    # Generate HTML for free-form text
    fft = free_form_text(df)
    html3 = free_form_text_html(fft)
    
    # Generate HTML for datatype mismatch
    ddm = datatype_mismatch(df)
    html4 = datatype_mismatch_html(ddm)
    
    # Generate HTML for column outliers
    co = column_outliers(df)
    html5 = column_outliers_html(co)
    
    # Generate HTML for potential misfielding
    misf = misfielded(df)
    html6 = misfielded_html(misf)
    
    # Generate recommendations
    n_missing = len(missing_df)
    rec1 = f"<li>There are {n_missing} columns with missing values in the dataset. Review missing values table below to assess severity.</li>"
    
    n_dup_col = len(dup_df)
    rec2 = f"<li>{dup_row_cnt} rows are duplicated over all columns. There are {n_dup_col} columns with duplicate values in the dataset. Review duplicates table below.</li>"
    
    n_fft = len(fft[fft["Percentage of Rows with Free-form Text"] > 0.0])
    rec3 = f"<li>There are {n_fft} columns with free form text. Review free-form-text table below for handling decisions.</li>"
    
    n_ddm = len(ddm)
    rec4 = f"<li>There are {n_ddm} columns with datatype mismatch in the dataset. These columns have a datatype that is different from majority datatype of cell values. Review datatype mismatch table below.</li>"
    
    n_co = len(co)
    rec5 = f"<li>There are {n_co} columns with outliers in the dataset. For numeric columns this refers to numeric value, for non-numeric columns this refers to value length. Refer to misfielded values table for handling decision.</li>"
    
    n_misf = len(misf)
    rec6 = f"<li>There are {n_misf} columns with potential misfielding/incorrect values in the dataset. Refer to misfielding tables below to verify these entries.</li>"
    
    # Combine HTML elements into a single document
    merged_html = f"""
        <html>
            <head>
            <h1>Data Profiling Report</h1>
            </head>
            <body>
                <h2>Recommendations</h2>
                <ul>
                    {rec1}
                    {rec2}
                    {rec3}
                    {rec4}
                    {rec5}
                    {rec6}
                <h2>Anomolous Conditions Detected</h2
                </ul>
                {html1}
                {html2}
                {html3}
                {html4}
                {html5}
                {html6}
            </body>
        </html>
    """
    
    with open('merged_output.html', 'w') as f:
        f.write(merged_html)



In [107]:
'''
routes files to appropriate block depending on file type. 

eg. a structured csv is sent to the structured block. 
a semi structured json is sent to semi structured block, etc.

'''
def file_routing(filetype_dict, data_path):
    if filetype_dict['classification'] == 'structured':
        structured_block(data_path,filetype_dict['extension'])
    #if filetype_dict['classification = semi-structured']:
    #    semistructured_block(data_path,filetype_dict['extension'])
    #if filetype_dict['classification = unstructured']:
    #    unstructured_block(data_path,filetype_dict['extension'])
    
    

In [108]:
#main code block 

data_path = "/Users/syedhadi/Desktop/datasets/mngr.csv"
info=check_input(data_path)
file_routing(info, data_path)